### Random Forest

In this notebook we are going to implement the Random Forest Algorithm. This algorithm is based on the Decision Tree algorithim implemented [here](https://github.com/CrispenGari/ml-algorithms/blob/main/09_decisiontree/01_Decision_Tree.ipynb)

<img src="https://www.tibco.com/sites/tibco/files/media_entity/2021-05/random-forest-diagram.svg" width="100%" alt=""/>


First we are going to create a Decision Tree algorithm and based on that, we will expand it to a random forest classifier.

### Basic imports



In [1]:
from collections import Counter
import numpy as np

### The `entropy` global function
Entropy is the measure of uncertainty of a random variable, it characterizes the impurity of an arbitrary collection of examples. The higher the entropy the more the information content.

![img](https://www.geeksforgeeks.org/wp-content/ql-cache/quicklatex.com-4af22a0bc64a8814e17e83fa12a05eef_l3.svg)


In [2]:
def entropy(y):
  hist = np.bincount(y)
  ps = hist / len(y)
  return -np.sum([p * np.log2(p) for p in ps if p > 0])

### The `Node` helper class

In [3]:
class Node:
  def __init__(
      self, feature=None, threshold=None, left=None, right=None, *, value=None
  ):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

  def is_leaf_node(self):
    return self.value is not None

### Decision Tree Algorithim

In [4]:
class DecisionTree:
  def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.n_feats = n_feats
    self.root = None

  def fit(self, X, y):
    """
    In the fit method of a decision tree we will grow the tree
    """
    self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
    self.root = self._grow_tree(X, y)

  def predict(self, X):
    """
    In the predict method we traverse down the treee
    """
    return np.array([self._traverse_tree(x, self.root) for x in X])

  def _grow_tree(self, X, y, depth=0):
    n_samples, n_features = X.shape
    n_labels = len(np.unique(y))
    # stopping criteria
    if (
        depth >= self.max_depth
        or n_labels == 1
        or n_samples < self.min_samples_split
    ):
        leaf_value = self._most_common_label(y)
        return Node(value=leaf_value)

    feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)
    # greedily select the best split according to information gain
    best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
    # grow the children that result from the split
    left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
    left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
    right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
    return Node(best_feat, best_thresh, left, right)

  def _best_criteria(self, X, y, feat_idxs):
    best_gain = -1
    split_idx, split_thresh = None, None
    for feat_idx in feat_idxs:
      X_column = X[:, feat_idx]
      thresholds = np.unique(X_column)
      for threshold in thresholds:
        gain = self._information_gain(y, X_column, threshold)
        if gain > best_gain:
          best_gain = gain
          split_idx = feat_idx
          split_thresh = threshold
    return split_idx, split_thresh

  def _information_gain(self, y, X_column, split_thresh):
    # parent loss
    parent_entropy = entropy(y)
    # generate split
    left_idxs, right_idxs = self._split(X_column, split_thresh)
    if len(left_idxs) == 0 or len(right_idxs) == 0:
      return 0
    # compute the weighted avg. of the loss for the children
    n = len(y)
    n_l, n_r = len(left_idxs), len(right_idxs)
    e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
    child_entropy = (n_l / n) * e_l + (n_r / n) * e_r
    # information gain is difference in loss before vs. after split
    ig = parent_entropy - child_entropy
    return ig

  def _split(self, X_column, split_thresh):
    left_idxs = np.argwhere(X_column <= split_thresh).flatten()
    right_idxs = np.argwhere(X_column > split_thresh).flatten()
    return left_idxs, right_idxs

  def _traverse_tree(self, x, node):
    if node.is_leaf_node():
      return node.value

    if x[node.feature] <= node.threshold:
      return self._traverse_tree(x, node.left)
    return self._traverse_tree(x, node.right)

  def _most_common_label(self, y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common


### Random Forest Classifier

First we are going to create a `bootstrap_sample` function that will randomply create samples for tree for us as well as the `most_comon_label` function that will return the most common label from an array of predicted labels.

In [5]:
def bootstrap_sample(X, y):
  n_samples = X.shape[0]
  idxs = np.random.choice(n_samples, n_samples, replace=True)
  return X[idxs], y[idxs]


def most_common_label(y):
  counter = Counter(y)
  most_common = counter.most_common(1)[0][0]
  return most_common


In [6]:
class RandomForest:
  def __init__(self, n_trees=10, min_samples_split=2, max_depth=100, n_feats=None):
      self.n_trees = n_trees
      self.min_samples_split = min_samples_split
      self.max_depth = max_depth
      self.n_feats = n_feats
      self.trees = []

  def fit(self, X, y):
    self.trees = []
    for _ in range(self.n_trees):
      tree = DecisionTree(
          min_samples_split=self.min_samples_split,
          max_depth=self.max_depth,
          n_feats=self.n_feats,
      )
      X_samp, y_samp = bootstrap_sample(X, y)
      tree.fit(X_samp, y_samp)
      self.trees.append(tree)

  def predict(self, X):
    tree_preds = np.array([tree.predict(X) for tree in self.trees])
    tree_preds = np.swapaxes(tree_preds, 0, 1)
    y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
    return np.array(y_pred)

  def accuracy(self, y_true, y_pred):
    return f"Accuracy: {np.equal(y_true, y_pred).sum()/len(y_true)*100}%"


### Testing the algorithm

We are going to load the dataset from `sklearn`.

In [7]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [8]:
data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [10]:
clf = RandomForest(n_trees=3, max_depth=10)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = clf.accuracy(y_test, y_pred)

In [11]:
acc

'Accuracy: 94.73684210526315%'

We can see that we have improved the accuracy by the use of the Random Forest over Decison Tree Algorithm. This is because instead of classifying a label with one tree we are classifying the label with multiple trees and take the most probable predicted label.

### Ref
2. [python engineer](https://github.com/python-engineer/MLfromscratch/blob/master/mlfromscratch/random_forest.py)